Importing the chromedriver file (saved on Desktop)

In [83]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

chromedriver = "/home/chintan/Desktop/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

In [84]:
# Opens a Chrome window and goes to google.com
driver = webdriver.Chrome(chromedriver)

In [85]:
#  Takes a specific webpage and returns a list of city names on that webpage - see comments to reduce this to US specific list 
def list_of_all_major_areas(url):
    list_of_all_major_cities=[]
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    list1=driver.find_elements_by_tag_name('a')
    for x in list1:
        list_of_all_major_cities.append(x.text)
    return(list_of_all_major_cities)
# l_cities=list_of_all_major_areas('https://www.craigslist.org/about/sites#US')
# l_cities_new=l_cities[14:434]
# l_cities_new.remove('mobile')

In [86]:
# Takes a list of city names and returns the list of cities worldwide -
def links_to_city_Cragslists(l_cities,url='https://www.craigslist.org/about/sites#US'):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    dict_cities={}
    for x in l_cities:
        element=driver.find_element_by_link_text(x)
        link=element.get_attribute('href')
        dict_cities[x]=link
    return(dict_cities)


In [87]:
# Takes a craigslist main link and returns the titles of all car post entries
def list_of_craigslist_en_from_address(url,click1='cars+trucks',click2='BY-OWNER ONLY'):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    location = driver.find_element_by_link_text(click1)
    location.click()
    time.sleep(1)
    location = driver.find_element_by_link_text(click2)
    location.click()
    time.sleep(1)
    total_count = driver.find_element_by_class_name('totalcount')
    no_of_pages=int(total_count.text)//120
    total_list=[]
    for x in range(no_of_pages):
        list_of_links = driver.find_elements_by_class_name('result-info')
        for x in list_of_links:
            total_list.append(x.text)
        time.sleep(1)    
        next_page=driver.find_element_by_xpath('//a[@title="next page"]')
        next_page.click()
        time.sleep(1)
    return(total_list)

In [88]:
# Takes a link from an inner Craigslist page and returns the list of attributes for that car/motorcycle
def attr_list_from_link(url):
    driver.get(url)
    list_of_b_tags=[]
    list1=driver.find_elements_by_tag_name('b')
    for x in list1:
        list_of_b_tags.append(x.text)
    list_of_spans=[]
    list2=driver.find_elements_by_tag_name('span')
    for y in list2:
        list_of_spans.append(y.text)
    attr_list=[]
    if len(list_of_b_tags)>1:
        for attr in list_of_b_tags:
            if attr in list_of_spans:
#                 attr_index_start=list_of_spans.index(attr)
                attr_index_start=len(list_of_spans) - 1 - list_of_spans[::-1].index(attr)
        attr_index_end=attr_index_start+len(list_of_b_tags)
        attr_list=list_of_spans[attr_index_start:attr_index_end]
        for string in list_of_spans:
            dollar=string.find('$')
            if dollar>=0:
                if len(string)<=7:
                    attr_list.append(string)
    return(attr_list)

In [89]:
# Takes a city Craislist site address, and unless otherwise specified, returns a list of href links for all cars posts by owners in it
def list_of_craigslist_href_from_address(url,click1='cars+trucks',click2='BY-OWNER ONLY'):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    location = driver.find_element_by_link_text(click1)
    location.click()
    time.sleep(1)
    location = driver.find_element_by_link_text(click2)
    location.click()
    time.sleep(1)
    total_count = driver.find_element_by_class_name('totalcount')
    no_of_pages=int(total_count.text)//120
    total_list=[]
    for x in range(no_of_pages):
        list_of_links = driver.find_elements_by_class_name('result-info')
        for x in list_of_links:
            strx=x.text
            si=strx.find('$')
            if si==-1:
                sib=strx.find('(')
                if sib==-1:
                    strx_cut=strx[8:]
                elif sib>=8:
                    strx_cut=strx[8:(sib-1)]
            elif si>=8:
                strx_cut=strx[8:(si-1)]
            #     print(strx_cut)
            continue_link = driver.find_element_by_partial_link_text(strx_cut)
            link=continue_link.get_attribute('href')
            total_list.append(link)
            print(link)
        # The list is saved as backup everytime this function is called upon and the backup is taken for every 120 entries
        with open('craig_url_list_details_here.pkl', 'wb') as picklefile:
            pkl.dump(total_list, picklefile)
        time.sleep(1)    
        next_page=driver.find_element_by_xpath('//a[@title="next page"]')
        next_page.click()
        time.sleep(1)
    return(total_list)

In [90]:
# carries out the entire process by calling the above functions as appropriate
# While the first two functions collect all Craigslist addresses across the US, I'm calling
import pickle as pkl

def Seattle():
    list_href=list_of_craigslist_href_from_address('https://seattle.craigslist.org')
    massive_list=[]
#     with open('craig_url_list_seattle_motorcycles_owner.pkl', 'rb') as picklefile: 
#         list_href = pkl.load(picklefile)
    for _ in range(2050,len(list_href)):
        temp=attr_list_from_link(list_href[_])
        massive_list.append(temp)
        if (len(massive_list)+1)%50==0:
            with open('scraped_craiglist_data_seattle_cars_owner.pkl', 'wb') as picklefile:
                pkl.dump(massive_list, picklefile)
        print(temp)

def Seattle_dealer():
    list_href=list_of_craigslist_href_from_address('https://seattle.craigslist.org',click1='cars+trucks',click2='BY-DEALER ONLY')
    massive_list=[]
#     with open('craig_url_list_seattle_motorcycles_owner.pkl', 'rb') as picklefile: 
#         list_href = pkl.load(picklefile)
    for _ in range(2050,len(list_href)):
        temp=attr_list_from_link(list_href[_])
        massive_list.append(temp)
        if (len(massive_list)+1)%50==0:
            with open('scraped_craiglist_data_seattle_cars_dealer.pkl', 'wb') as picklefile:
                pkl.dump(massive_list, picklefile)
        print(temp)

In [91]:
Seattle()

['1999 suzuki hayabusa', 'condition: excellent', 'engine displacement (CC): 1300', 'fuel: gas', 'odometer: 34061', 'title status: clean', 'transmission: manual', '$6000']
['2008 harley fatboy', 'condition: like new', 'engine displacement (CC): 1573', 'fuel: gas', 'odometer: 4145', 'paint color: black', 'title status: clean', 'transmission: manual', '$10250']
['2004 bmw r1150r rockster', 'condition: excellent', 'engine displacement (CC): 1150', 'fuel: gas', 'odometer: 11000', 'paint color: custom', 'title status: clean', 'transmission: manual', '$6750']
['2006 FXDLI Dyna Low Rider', 'condition: excellent', 'fuel: gas', 'odometer: 11500', 'paint color: black', 'title status: clean', 'transmission: manual', '$8750']
['2009 harley-davidson touring', 'fuel: gas', 'transmission: manual', '$20000']
['2009 Honda Rebel 250', 'condition: good', 'fuel: gas', 'odometer: 7200', 'paint color: black', 'title status: clean', 'transmission: manual', '$1700']
['1995 suzuki dr650se', 'condition: good', '

['1982 Buckle Tree, Sturgis, S.D.', 'VIN: 00000000', 'condition: new', 'engine displacement (CC): 29', 'fuel: other', 'odometer: 0000000', 'paint color: custom', 'title status: clean', 'transmission: other', '$1500']
['1996 kawasaki vulcan 1500 classic', 'condition: good', 'engine displacement (CC): 1500', 'fuel: gas', 'odometer: 47000', 'paint color: custom', 'title status: clean', 'transmission: manual', '$3300']
['2006 yamaha ttr 125', 'condition: good', 'engine displacement (CC): 125', 'fuel: gas', 'paint color: blue', 'title status: clean', 'transmission: manual', '$2000']
['leathers', 'fuel: other', 'transmission: other', '$75']
['2006 hd heritage softail FLSTCI', 'condition: good', 'fuel: gas', 'odometer: 20836', 'transmission: manual', '$7500']
['2000 suzuki katana 600', 'condition: excellent', 'engine displacement (CC): 600', 'fuel: gas', 'odometer: 13800', 'paint color: red', 'title status: clean', 'transmission: manual', '$1500']
['Harley Davidson', 'fuel: gas', 'transmissio

['2006 Honda', 'condition: good', 'engine displacement (CC): 450', 'fuel: gas', 'paint color: red', 'transmission: manual', '$2400']
['2011 Yamaha fz8', 'condition: like new', 'engine displacement (CC): 800', 'fuel: gas', 'paint color: black', 'title status: clean', 'transmission: manual', '$5500']
['2000 FLHPI', 'engine displacement (CC): 1450', 'fuel: gas', 'paint color: white', 'title status: clean', 'transmission: manual', '$4800']
['2012 Aprilia Dorsoduro', 'condition: excellent', 'engine displacement (CC): 1200', 'fuel: gas', 'odometer: 10364', 'title status: clean', 'transmission: manual', '$6995']
['honda ch250', 'fuel: gas', 'transmission: automatic', '$1000']
['Honda XR 250 R', 'condition: excellent', 'fuel: gas', 'paint color: red', 'transmission: manual', '$2450']
['2014 yamaha bolt', 'condition: like new', 'engine displacement (CC): 950', 'fuel: gas', 'odometer: 4000', 'paint color: green', 'title status: clean', 'transmission: manual', '$4000']
['2011 bmw k1300s', 'fuel: 

['2015 Kawasaki Ninja 650R', 'condition: excellent', 'fuel: gas', 'odometer: 9315', 'title status: clean', 'transmission: manual', '$5500']
['2015 kawasaki ninja 650r', 'condition: excellent', 'fuel: gas', 'odometer: 9315', 'title status: clean', 'transmission: manual', '$5500']
['1993 harley', 'condition: excellent', 'engine displacement (CC): 1340', 'fuel: gas', 'odometer: 34000', 'paint color: black', 'title status: clean', 'transmission: manual', '$9000']
['2005 Suzuki 125l', 'VIN: Js1df43b352101639', 'condition: excellent', 'engine displacement (CC): 124', 'fuel: gas', 'paint color: blue', 'title status: lien', 'transmission: manual', '$800']
['Yamaha Zuma.', 'condition: like new', 'engine displacement (CC): 49', 'fuel: gas', 'odometer: 738', 'paint color: blue', 'title status: clean', 'transmission: other', '$1600']
['1997 HONDA BLACKBIRD', 'condition: good', 'engine displacement (CC): 1137', 'fuel: gas', 'odometer: 72000', 'paint color: black', 'title status: clean', 'transmissi

['2006 yamaha vstar', 'condition: like new', 'engine displacement (CC): 1300', 'fuel: gas', 'odometer: 13840', 'paint color: red', 'title status: clean', 'transmission: manual', '$8000']
['Ssr', 'fuel: gas', 'transmission: manual', '$599']
['1981 yamaha 750 Virago', 'condition: excellent', 'fuel: gas', 'paint color: black', 'title status: clean', 'transmission: manual', '$1700']
['2006 suzuki katana', 'condition: good', 'engine displacement (CC): 750', 'fuel: gas', 'odometer: 8100', 'paint color: orange', 'title status: rebuilt', 'transmission: manual', '$2550']
['2007 Honda CRF-450', 'condition: excellent', 'fuel: gas', 'paint color: red', 'title status: clean', 'transmission: manual', '$3500']
['2007 Honda CRF-450', 'condition: excellent', 'fuel: gas', 'paint color: red', 'title status: clean', 'transmission: manual', '$3500']
['2007 Honda CRF-450', 'condition: excellent', 'fuel: gas', 'paint color: red', 'title status: clean', 'transmission: manual', '$3500']
['Vega', 'fuel: gas', '

['1986 harley davidson', 'condition: fair', 'fuel: electric', 'paint color: white', 'title status: clean', 'transmission: automatic', '$900']
['Baja Blitz', 'fuel: gas', 'transmission: other', '$375']
['2001 bmw r1150gs', 'condition: excellent', 'engine displacement (CC): 1150', 'fuel: gas', 'odometer: 26500', 'paint color: yellow', 'title status: clean', 'transmission: manual', '$5500']
['2016 Harley Davidson Low Rider S', 'fuel: gas', 'transmission: manual', '$18500']
['bmw r1200 rt', 'fuel: gas', 'transmission: manual', '$21000']
['2015 ktm 300 xc', 'condition: excellent', 'fuel: gas', 'title status: clean', 'transmission: manual', '$6700']
['1986 Kawaski Concours', 'fuel: gas', 'transmission: manual', '$1950']
['ktm 250sx', 'fuel: gas', 'transmission: manual', '$5000']
['2005 Suzuki DRZ 125 Dirt Bike 2005', 'condition: excellent', 'engine displacement (CC): 125', 'fuel: gas', 'paint color: blue', 'title status: clean', 'transmission: manual', '$1500']
['Honda', 'fuel: gas', 'paint 

['1983 honda shadow 750', 'condition: good', 'fuel: gas', 'transmission: manual', '$950']


In [ ]:
Seattle_dealer()